In [ ]:
!pip uninstall cudf-cu12 ibis-framework bigframes -y


Found existing installation: bigframes 1.15.0
Uninstalling bigframes-1.15.0:
  Successfully uninstalled bigframes-1.15.0


In [ ]:
!pip install pyarrow==17.0.0



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install datasets --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00


In [ ]:
!pip list | grep -E 'pyarrow|datasets'


datasets                         2.21.0
pyarrow                          17.0.0
pyarrow-hotfix                   0.6
tensorflow-datasets              4.9.6
vega-datasets                    0.9.0


In [ ]:
from datasets import load_dataset

# Load the dataset with the 'pqa_labeled' configuration
dataset = load_dataset('pubmed_qa', 'pqa_labeled')

# Print dataset structure
print("Dataset Structure:\n", dataset)

# Access specific splits of the dataset
print("\nTrain Split Example:\n", dataset['train'][0])

# Access all the keys and columns available in the dataset
print("\nDataset Columns and Features:\n", dataset['train'].column_names)

# Access metadata and data types of columns
print("\nDataset Features and Data Types:\n", dataset['train'].features)

# Display a sample of the dataset
sample_size = 5
print(f"\nSample of {sample_size} rows from Train Split:")
for i in range(sample_size):
    print(f"\nSample {i+1}: {dataset['train'][i]}")

Dataset Structure:
 DatasetDict({
    train: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 1000
    })
})

Train Split Example:
 {'pubid': 21645374, 'question': 'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?', 'context': {'contexts': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.', 'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo

In [ ]:
from datasets import load_dataset

# Load the dataset with the 'pqa_labeled' configuration
dataset = load_dataset('pubmed_qa', 'pqa_labeled')

# Define a function to transform the dataset into the desired format
def transform_pubmed_qa(example):
    # Access and strip the 'question' field
    human_text = example['question'].strip()

    # Access the 'long_answer' field and handle potential None values
    long_answer_text = example['long_answer'].strip() if example['long_answer'] else ''

    # Use 'long_answer' as the assistant's response
    assistant_text = long_answer_text

    # Apply the new template with <s>[INST] ... [/INST] ... </s>
    reformatted_text = f'<s>[INST] {human_text} [/INST] {assistant_text} </s>'

    return {'text': reformatted_text}

# Apply the transformation to the 'train' split and remove other columns
transformed_dataset = dataset['train'].map(
    transform_pubmed_qa,
    remove_columns=dataset['train'].column_names
)

# Display a sample of the transformed dataset
sample_size = 5
for i in range(sample_size):
    print(f"\nTransformed Sample {i+1}: {transformed_dataset[i]['text']}")



Transformed Sample 1: <s>[INST] Do mitochondria play a role in remodelling lace plant leaves during programmed cell death? [/INST] Results depicted mitochondrial dynamics in vivo as PCD progresses within the lace plant, and highlight the correlation of this organelle with other organelles during developmental PCD. To the best of our knowledge, this is the first report of mitochondria and chloroplasts moving on transvacuolar strands to form a ring structure surrounding the nucleus during developmental PCD. Also, for the first time, we have shown the feasibility for the use of CsA in a whole plant system. Overall, our findings implicate the mitochondria as playing a critical and early role in developmentally regulated PCD in the lace plant. </s>

Transformed Sample 2: <s>[INST] Landolt C and snellen e acuity: differences in strabismus amblyopia? [/INST] Using the charts described, there was only a slight overestimation of visual acuity by the Snellen E compared to the Landolt C, even in

In [ ]:
transformed_dataset[0]

{'text': '<s>[INST] Do mitochondria play a role in remodelling lace plant leaves during programmed cell death? [/INST] Results depicted mitochondrial dynamics in vivo as PCD progresses within the lace plant, and highlight the correlation of this organelle with other organelles during developmental PCD. To the best of our knowledge, this is the first report of mitochondria and chloroplasts moving on transvacuolar strands to form a ring structure surrounding the nucleus during developmental PCD. Also, for the first time, we have shown the feasibility for the use of CsA in a whole plant system. Overall, our findings implicate the mitochondria as playing a critical and early role in developmentally regulated PCD in the lace plant. </s>'}

In [ ]:
# Save the transformed dataset to a file
transformed_dataset.save_to_disk("/content/drive/MyDrive/Colab Notebooks/CHAT_BOT_APP/pbmed_dataset")


Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset, DatasetDict, load_from_disk
dataset = load_from_disk('/content/drive/MyDrive/Colab Notebooks/CHAT_BOT_APP/pbmed_dataset')

dataset[0]

{'text': '<s>[INST] Do mitochondria play a role in remodelling lace plant leaves during programmed cell death? [/INST] Results depicted mitochondrial dynamics in vivo as PCD progresses within the lace plant, and highlight the correlation of this organelle with other organelles during developmental PCD. To the best of our knowledge, this is the first report of mitochondria and chloroplasts moving on transvacuolar strands to form a ring structure surrounding the nucleus during developmental PCD. Also, for the first time, we have shown the feasibility for the use of CsA in a whole plant system. Overall, our findings implicate the mitochondria as playing a critical and early role in developmentally regulated PCD in the lace plant. </s>'}

In [ ]:
from huggingface_hub import login

login(token='Your api token')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Push the transformed dataset to Hugging Face

transformed_dataset.push_to_hub("viniljpk/pubmedQA_v1")  # Replace with your repository name

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/viniljpk/pubmedQA_v1/commit/7024bb1e59988033857ac977f472ab93e23dbf62', commit_message='Upload dataset', commit_description='', oid='7024bb1e59988033857ac977f472ab93e23dbf62', pr_url=None, pr_revision=None, pr_num=None)